# CPS_SQL_01_Fundamentals

Pure SQL practice using DuckDB inside Jupyter.


## Setup (run once)
Connect to DuckDB and load CSVs as tables.

In [ ]:
%load_ext sql
%sql duckdb:///sql/practice.duckdb

In [ ]:
%%sql
CREATE OR REPLACE TABLE employees AS SELECT * FROM read_csv_auto('sql/data/employee_data_large.csv');
CREATE OR REPLACE TABLE sales AS SELECT * FROM read_csv_auto('sql/data/sales_large.csv');
CREATE OR REPLACE TABLE students AS SELECT * FROM read_csv_auto('sql/data/student_data_large.csv');
CREATE OR REPLACE TABLE departments AS SELECT * FROM read_csv_auto('sql/data/departments.csv');
CREATE OR REPLACE TABLE customers AS SELECT * FROM read_csv_auto('sql/data/customers.csv');
CREATE OR REPLACE TABLE orders AS SELECT * FROM read_csv_auto('sql/data/orders.csv');

## Q1
Select all columns from `employees`.

In [ ]:
%%sql
-- write your query here


## Q2
Show average salary by department.

In [ ]:
%%sql
-- write your query here


## Q3
Count employees by status.

In [ ]:
%%sql
-- write your query here


## Q4
Find the top 5 highest-paid employees.

In [ ]:
%%sql
-- write your query here


## Q5
Identify rows with missing salary or age.

In [ ]:
%%sql
-- write your query here
